### Using Tree data

In [ ]:
import pandas as pd
from sklearn import gaussian_process
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

dendro = pd.read_feather("data/17766_12.feather")
dendro.dendroNr = (
    dendro[["dendroNr", "year"]].astype(str).agg("_".join, axis=1)
)  # Merge tree and year
dendro

In [ ]:
from sklearn.gaussian_process.kernels import RBF

tree_models = [
    gaussian_process.GaussianProcessRegressor(
        0.5 ** 2 * RBF(100, (1e-3, 1e8)), n_restarts_optimizer=1
    )
    for _ in range(3)
]

X = np.array(range(183)).reshape(-1, 1)

for i, tree_model in enumerate(tree_models):
    filt = dendro[
        dendro.species.eq(pd.unique(dendro.species)[i]) & dendro.DOY.mod(30).eq(0)
        ]  # Filter to prevent overfitting to data mean
    tree_model.fit(filt.DOY.array.reshape(-1, 1), filt.growth - filt.growth.mean())
    print(tree_model.kernel_)

In [ ]:
fig = plt.figure(figsize=(16, 9))
sns.set_theme(style="whitegrid", palette="pastel")

# All lines
sns.lineplot(
    data=dendro,
    x="DOY",
    y="growth",
    style="species",
    units="dendroNr",
    lw=0.8,
    estimator=None,
    c="lightgray",
    alpha=0.5,
)

plt.gca().set(xlim=(0, 182))

# Plot means
sns.lineplot(
    data=dendro,
    x="DOY",
    y="growth",
    hue="species",
    # estimator=None,
    # errorbar=('ci', 99),
    errorbar=None,
    lw=1,
    zorder=2,
    alpha=0.8,
)

X_test = np.linspace(0, 182, 300, True)
# Plot GP
for i, tree_model in enumerate(tree_models):
    filt = dendro[dendro.species.eq(pd.unique(dendro.species)[i]) & dendro.DOY.mod(30).eq(0)]
    sns.lineplot(
        x=X_test,
        y=tree_model.predict(X_test.reshape(-1, 1)).ravel() + filt.growth.mean(),
        lw=4,
        zorder=3,
        color=sns.color_palette()[i if i == 0 else [2, 1][i - 1]],
    )

plt.show()